In [132]:
# populate_tables.py
import sqlalchemy as db

import random
# Set up connections between sqlalchemy and postgres dbapi
engine = db.create_engine("sqlite:///burger.db")
connection = engine.connect()

# Instantiate metadata class
metadata = db.MetaData()
# Instantiate faker class
from faker import Faker
from faker_food import FoodProvider
faker = Faker()
faker.random.seed()
faker.add_provider(FoodProvider)
# Reflect metadata/schema from existing postgres database
with engine.connect() as conn:
    metadata.reflect(conn)

In [133]:
table_dict = {}
for t in metadata.sorted_tables:
    table_dict[f"{t.name}"] = metadata.tables[f"{t.name}"]

Table pays

In [17]:
with engine.begin() as conn:
    for _ in range(5):
        insert_stmt = table_dict["Pays"].insert().values(
        pays = faker.country()
        )
        conn.execute(insert_stmt)

Table restaurant

In [28]:
pays_pk = connection.execute(db.select([table_dict["Pays"].c.pays])).fetchall()

with engine.begin() as conn:
    for _ in range(50):
      insert_stmt = table_dict["Restaurant"].insert().values(
        code_postal = faker.postalcode(),
        pays = random.choice(pays_pk)[0],
        capacité = faker.pyint(0,500),
        espace_enfant = faker.pyint(0,1),
        service_rapide = faker.pyint(0,1),
        accessibilité = faker.pyint(0,1),
        parking = faker.pyint(0,200),
      )
      conn.execute(insert_stmt)

Table employe

Comme les employés ont besoin de supérieur on ne peut pas les créer tous d'un coup. On commence donc par créer les directeurs, les managers puis enfin 

In [33]:
restaurant_pk = connection.execute(db.select([table_dict["Restaurant"].c.code_postal])).fetchall()


# Create Directors
with engine.begin() as conn:
    for restaurant in restaurant_pk:
      insert_stmt = table_dict["Employe"].insert().values(
        code_postal = restaurant[0],
        poste = "directeur",
        experience = faker.pyint(0,40),
        nom = faker.unique.name(),
        adresse = faker.unique.address(),
        note = faker.pyint(0,10)
      )
      conn.execute(insert_stmt)

In [34]:
# Create Managers

directors_pk = connection.execute(db.select([table_dict["Employe"].c.id_employe])).fetchall()

with engine.begin() as conn:
    for restaurants in range(250):
      insert_stmt = table_dict["Employe"].insert().values(
        code_postal = random.choice(restaurant_pk)[0],
        id_superieur = random.choice(directors_pk)[0],
        poste = "manager",
        experience = faker.pyint(0,40),
        nom = faker.unique.name(),
        adresse = faker.unique.address(),
        note = faker.pyint(0,10)
      )
      conn.execute(insert_stmt)

In [43]:
# Create Employe

managers_pk = connection.execute(db.select([table_dict["Employe"].c.id_employe]).where(table_dict["Employe"].c.poste == "manager")).fetchall()

with engine.begin() as conn:
    for restaurants in range(2500):
      insert_stmt = table_dict["Employe"].insert().values(
        code_postal = random.choice(restaurant_pk)[0],
        id_superieur = random.choice(managers_pk)[0],
        poste = random.choice(["cuisinier","vendeur"]),
        experience = faker.pyint(0,40),
        nom = faker.unique.name(),
        adresse = faker.unique.address(),
        note = faker.pyint(0,10)
      )
      conn.execute(insert_stmt)

Table rib

In [52]:
employe_list = connection.execute(db.select(
    table_dict["Employe"].c.id_employe,
    table_dict["Employe"].c.nom,
    table_dict["Employe"].c.adresse
    )).fetchall()

with engine.begin() as conn:
    for employe in employe_list:
      insert_stmt = table_dict["RIB"].insert().values(
        id_employe = employe[0],
        iban = faker.iban(),
        bic = faker.bban(),
        proprietaire = employe[1],
        adresse = employe[2]
      )
      conn.execute(insert_stmt)


Table paie

In [65]:
employe_list = connection.execute(db.select(
    table_dict["Employe"].c.id_employe,
    table_dict["Employe"].c.experience,
    table_dict["Employe"].c.note,
    table_dict["Employe"].c.poste
    )).fetchall()

with engine.begin() as conn:
    # On travaille employé par employé
    for employe in employe_list:
        # On génére un historique en fonction de l'experience
        for annee in range(employe[1]):
            for month in range(1,13):
                insert_stmt = table_dict["Paie"].insert().values(
                    id_employe = employe[0],
                    date = f"{2023-annee}-{month}",
                    # Le salaire dépend de l'expériencet de la note
                    salaire_net = 2000 + 20 * employe[1] + 30 * employe[2]
                )
                conn.execute(insert_stmt)

Gestion des plats

In [ ]:
item_table = db.Table('Item', metadata,
              db.Column('nom_item', db.String(), primary_key=True),
              db.Column('type', db.String(), nullable=False),
              db.Column('prix', db.Float(), nullable=False),
              )


In [88]:
dish_list=[]
for _ in range(100):
    dish_list.append(faker.dish())

unique_dish = list(set(dish_list))

In [89]:
# l'argument unique ne semble pas marcher avec le provider importer d'un autre package,
# on trouve donc une astuce pour avoir un liste unique

with engine.begin() as conn:
    for i in range(len(unique_dish)):
      insert_stmt = table_dict["Item"].insert().values(
        nom_item = unique_dish[i],
        type = random.choice(["dessert","plat","boisson"]),
        prix = faker.pyfloat(left_digits=2, right_digits=2,  min_value=2.0, max_value=20.0)
      )
      conn.execute(insert_stmt)

Création des Menu

In [134]:
boisson_pk = connection.execute(db.select([table_dict["Item"].c.nom_item]).where(table_dict["Item"].c.type == "boisson")).fetchall()
plat_pk = connection.execute(db.select([table_dict["Item"].c.nom_item]).where(table_dict["Item"].c.type == "boisson")).fetchall()
dessert_pk = connection.execute(db.select([table_dict["Item"].c.nom_item]).where(table_dict["Item"].c.type == "boisson")).fetchall()

# id_unique_menu = 0

with engine.begin() as conn:
    for _ in range(20):
      insert_stmt = table_dict["Menu"].insert().values(
        # id_menu = id_unique_menu,
        boisson = random.choice(boisson_pk)[0],
        plat = random.choice(plat_pk)[0],
        dessert = random.choice(dessert_pk)[0],
        prix = faker.pyfloat(left_digits=2, right_digits=2,  min_value=12.0, max_value=35.0)

      )
      conn.execute(insert_stmt)
      # id_unique_menu+=1

Creation des ingrédients

In [92]:
ingredient_list=[]
for _ in range(100):
    ingredient_list.append(faker.ingredient())

unique_ingredient = list(set(ingredient_list))

In [95]:
with engine.begin() as conn:
    for i in range(len(unique_ingredient)):
      insert_stmt = table_dict["Ingredient"].insert().values(
        nom_ingredient = unique_ingredient[i],
        cout = faker.pyfloat(left_digits=2, right_digits=2,  min_value=1.0, max_value=10.0)
      )
      conn.execute(insert_stmt)

Gestion des recettes

In [ ]:
dish_pk = connection.execute(db.select([table_dict["Item"].c.nom_item])).fetchall()
ingredient_pk = connection.execute(db.select([table_dict["Ingredient"].c.nom_ingredient])).fetchall()


with engine.begin() as conn:
    for i in range(len(dish_pk)):
        ingredient_selectionne = []
        for nb_ingredient in range(random.randrange(1,6)):
            selected_ingredient = random.choice([e for e in ingredient_pk if e[0] not in ingredient_selectionne])[0]
            ingredient_selectionne.append(selected_ingredient)
            
            insert_stmt = table_dict["Recette"].insert().values(
                nom_item = dish_pk[i][0],
                nom_ingredient = selected_ingredient
                
            )
            conn.execute(insert_stmt)

Gestion des bills

In [147]:
restaurant_pk = connection.execute(db.select([table_dict["Restaurant"].c.code_postal])).fetchall()


with engine.begin() as conn:
    for i in range(len(restaurant_pk)):
        vendeur_pk = connection.execute(db.select([table_dict["Employe"].c.id_employe])\
            .where(table_dict["Employe"].c.poste == "vendeur")\
            .where(table_dict["Employe"].c.code_postal == restaurant_pk[i][0]))\
            .fetchall()

        for _ in range(len(vendeur_pk)*10):
            insert_stmt = table_dict["Bill"].insert().values(
                code_postal = restaurant_pk[i][0],
                id_employe = random.choice(vendeur_pk)[0],
                borne = 0,
                moyen_paiement = random.choice(["CB","cash","cheque"]),
                prix_total = faker.pyfloat(min_value=10.0, max_value=150.0)    
                )
            conn.execute(insert_stmt)

Panier Item

In [154]:
item_pk = connection.execute(db.select([table_dict["Item"].c.nom_item])).fetchall()
print( len(item_pk) )
print( len(set(item_pk)) )

36
36


In [157]:
item_pk = connection.execute(db.select([table_dict["Item"].c.nom_item])).fetchall()
bill_pk = connection.execute(db.select([table_dict["Bill"].c.id_bill])).fetchall()


with engine.begin() as conn:
    for i in range(len(bill_pk)):
        item_selectionne_list = []
        for _ in range(random.randrange(1,10)):
            selected_item = random.choice([e for e in item_pk if e[0] not in item_selectionne_list])[0]
            item_selectionne_list.append(selected_item)
            insert_stmt = table_dict["PanierItemTable"].insert().values(
                id_bill = bill_pk[i][0],
                nom_item = selected_item,
                quantite = faker.pyint(0,10)
            )
            conn.execute(insert_stmt)

In [160]:
menu_pk = connection.execute(db.select([table_dict["Menu"].c.id_menu])).fetchall()
bill_pk = connection.execute(db.select([table_dict["Bill"].c.id_bill])).fetchall()


with engine.begin() as conn:
    for i in range(len(bill_pk)):
        menu_selectionne_list = []
        for _ in range(random.randrange(1,5)):
            selected_menu = random.choice([e for e in menu_pk if e[0] not in menu_selectionne_list])[0]
            menu_selectionne_list.append(selected_menu)
            insert_stmt = table_dict["PanierMenuTable"].insert().values(
                id_bill = bill_pk[i][0],
                id_menu = selected_menu,
                quantite = faker.pyint(1,5)
            )
            conn.execute(insert_stmt)

carte menu

In [161]:
menu_pk = connection.execute(db.select([table_dict["Menu"].c.id_menu])).fetchall()
pays_pk = connection.execute(db.select([table_dict["Pays"].c.pays])).fetchall()


with engine.begin() as conn:
    for i in range(len(pays_pk)):
        random_menu = random.sample(menu_pk, 10)
        for j in range(len(random_menu)):
            insert_stmt = table_dict["CarteMenu"].insert().values(
                pays = pays_pk[i][0],
                id_menu = random_menu[j][0]
            )
            conn.execute(insert_stmt)

In [164]:
item_pk = connection.execute(db.select([table_dict["Item"].c.nom_item])).fetchall()
pays_pk = connection.execute(db.select([table_dict["Pays"].c.pays])).fetchall()


with engine.begin() as conn:
    for i in range(len(pays_pk)):
        random_item = random.sample(item_pk, 20)
        for j in range(len(random_item)):
            insert_stmt = table_dict["CarteItem"].insert().values(
                pays = pays_pk[i][0],
                nom_item = random_item[j][0]
            )
            conn.execute(insert_stmt)

Gestion des Stocks

In [166]:
ingredient_pk = connection.execute(db.select([table_dict["Ingredient"].c.nom_ingredient])).fetchall()
restaurant_pk = connection.execute(db.select([table_dict["Restaurant"].c.code_postal])).fetchall()


with engine.begin() as conn:
    for i in range(len(restaurant_pk)):
        for j in range(len(ingredient_pk)):
            insert_stmt = table_dict["Stock"].insert().values(
                nom_ingredient = ingredient_pk[j][0],
                code_postal = restaurant_pk[i][0],
                quantite = faker.pyint(0,200)
            )
            conn.execute(insert_stmt)